In [ ]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 17.0 MB/s eta 0:00:00


In [ ]:
import pymongo

In [ ]:
client=pymongo.MongoClient("Paste_MongoDB_URL")
db=client.sample_mflix
collection=db.movies

In [ ]:
items1=collection.find().limit(5)
items1

In [ ]:
for item in items1:
  print(item)

{'_id': ObjectId('573a1390f29313caabcd42e8'), 'plot': 'A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels.', 'genres': ['Short', 'Western'], 'runtime': 11, 'cast': ['A.C. Abadie', "Gilbert M. 'Broncho Billy' Anderson", 'George Barnes', 'Justus D. Barnes'], 'poster': 'https://m.media-amazon.com/images/M/MV5BMTU3NjE5NzYtYTYyNS00MDVmLWIwYjgtMmYwYWIxZDYyNzU2XkEyXkFqcGdeQXVyNzQzNzQxNzI@._V1_SY1000_SX677_AL_.jpg', 'title': 'The Great Train Robbery', 'fullplot': "Among the earliest existing films in American cinema - notable as the first film that presented a narrative story to tell - it depicts a group of cowboy outlaws who hold up a train and rob the passengers. They are then pursued by a Sheriff's posse. Several scenes have color included - all hand tinted.", 'languages': ['English'], 'released': datetime.datetime(1903, 12, 1, 0, 0), 'directors': ['Edwin S. Porter'], 'rated': 'TV-G', 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'},

In [ ]:
import requests

In [ ]:
hf_token="huggin_face_token"
embedding_url="https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

def generate_embedding(text):
  response=requests.post( embedding_url,
                         headers={"Authorization": f"Bearer {hf_token}"},
                         json={"inputs":text})
  if response.status_code!=200:
    raise ValueError(f"Request failed within the status code {response.status_code}:{response.text}")
  return response.json()

print(generate_embedding("Hello World!"))

[-0.020386826246976852, 0.025280892848968506, -0.0005662075709551573, 0.01161543931812048, -0.03798838332295418, -0.1199813112616539, 0.041709478944540024, -0.020857160910964012, -0.0590067058801651, 0.024232449010014534, 0.062120165675878525, 0.06767994910478592, 0.03310021385550499, -0.010369363240897655, -0.031215645372867584, -0.032733254134655, -0.0021118083968758583, 0.009261918254196644, -0.12476454675197601, 0.011236821301281452, 0.039045341312885284, 0.05440257862210274, -0.00282548856921494, 0.04455629363656044, -0.08542020618915558, -0.02287374623119831, 0.03914057835936546, 0.036046870052814484, -0.03212675824761391, -0.06425870209932327, 0.05812910199165344, 0.046690814197063446, 0.0806155875325203, -0.007734257262200117, -0.022083153948187828, 0.06713154911994934, -0.04504144564270973, -0.1021212711930275, 0.0012644159141927958, 0.04680195823311806, 0.026395881548523903, -0.06990952789783478, -0.04453347995877266, -0.006901935674250126, 0.01928866095840931, 0.020590808242

In [ ]:
for doc in collection.find({'plot':{"$exists": True}}).limit(50):
  doc['plot_embedding_hf']=generate_embedding(doc['plot'])
  collection.replace_one({'_id':doc['_id']},doc)

In [ ]:
query = "british"

results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": generate_embedding(query),
    "path": "plot_embedding_hf",
    "numCandidates": 100,
    "limit": 4,
    "index": "semantic_plot",
      }}
]);

for document in results:
    print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')

Movie Name: Robin Hood,
Movie Plot: A nobleman becomes the vigilante Robin Hood who protects the oppressed English people from the tyrannical Prince John.

Movie Name: Beau Geste,
Movie Plot: Michael "Beau" Geste leaves England in disgrace and joins the infamous French Foreign Legion. He is reunited with his two brothers in North Africa, where they face greater danger from their...

Movie Name: The Strong Man,
Movie Plot: A meek Belgian soldier (Harry Langdon) fighting in World War I receives penpal letters and a photo from "Mary Brown", an American girl he has never met. He becomes infatuated with her by ...

Movie Name: The Four Horsemen of the Apocalypse,
Movie Plot: An extended family split up in France and Germany find themselves on opposing sides of the battlefield during World War I.

